# Código para importar series macro del INE, BDE, ECB Data wharehouse

NOTAS: PARA ECB solo series mensuales

1. Importamos las librerias a utilizar

In [1]:
import requests
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date
import locale
import io

2. Datos a modificar, añadir series nuevas a importar a la lista si hace falta. 
Poner lista y tabla (datos de tabla no vienen en el request)

In [2]:
url_datos = 'http://servicios.ine.es/wstempus/js/ES/DATOS_SERIE/{codigo}?date={FECHA_INI}:{FECHA_FIN}'
url_metadatos = 'https://servicios.ine.es/wstempus/js/ES/SERIE/{codigo}'
url_ecb = 'https://sdw-wsrest.ecb.europa.eu/service/data/{FLOWREF}/{KEY}?format=csvdata'

FECHA_INI = '19900101'
FECHA_FIN = date.today().strftime("%Y%m%d")

In [3]:
# Formato "SERIE","TABLA"
LISTA_SERIES_INE = [['CNTR4851','30681'],
                    ['CNTR4892','30681'],
                    ['CNTR4893','30681'],
                    ['EPA86913','4247'],
                    ['IPC251852','50902'],
                    ['IPCA1847','23706'],
                    ['IPC251856','50902'],
                    ['IPCA1885','23706'],
                    ['IPV769','25171'],
                    ['IPV948','25171'],
                    ['IPV949','25171'],
                    ['IPI10277','26061'],
                    ['IPI10276','26061'],
                    ['ICNE91','27142'],
                    ['ICNE90','27142'],
                    ['ICNE89','27142']]

# Series BDE (número sequencial)
LISTA_SERIES_BDE = ['2333578','2333579','2333591','2333592','3833168','2806199','989934','989935','989243',
                   '989244','2806196','989936','3570404','3570405','3777293','3779116']

# Tablas BDE
TABLAS_SERIES_BDE = ['https://www.bde.es/webbde/es/estadis/infoest/series/be0413.csv',
                    'https://www.bde.es/webbde/es/estadis/infoest/series/be0104.csv',
                    'https://www.bde.es/webbde/es/estadis/infoest/series/be1908.csv',
                    'https://www.bde.es/webbde/es/estadis/infoest/series/be1904.csv',
                    'https://www.bde.es/webbde/es/estadis/infoest/series/be1903.csv',
                    'https://www.bde.es/webbde/es/estadis/infoest/series/be1909.csv',
                    'https://www.bde.es/webbde/es/estadis/infoest/series/be1601.csv',
                    'https://www.bde.es/webbde/es/estadis/infoest/series/be1602.csv']

# Tablas ECB [FLOWREF,KEY] (solo series mensuales de momento)
# Ejemplo https://sdw.ecb.europa.eu/quickview.do?SERIES_KEY=143.FM.M.US.USD.DS.EI.S_PCOMP.HSTA --> 
#         FLOWREF --> FM  KEY --> M.US.USD.DS.EI.S_PCOMP.HSTA

LISTA_SERIES_ECB = [['FM','M.U2.EUR.RT.MM.EURIBOR3MD_.HSTA'],
                    ['FM','M.US.USD.DS.EI.S_PCOMP.HSTA'],
                    ['EXR','M.GBP.EUR.SP00.A']]

# Comentarios series

COMENTARIOS_SERIES = [['2333578', 'Serie stock crédito EMP'],
                      ['2333579','Serie stock crédito PART'],
                      ['2333591','Serie stock crédito NPE EMP'],
                      ['2333592','Serie stock crédito NPE Part'],
                      ['3777293','Serie Renta Bruta Disponible BDE'],
                      ['3570404','Serie stock crédito PART Consumo'],
                      ['3570405','Serie stock crédito NPE PART Consumo'],
                      ['IPC251852','Índice de precios de consumo'],
                      ['IPC251856','Índice de precios de consumo'],
                      ['IPCA1847','Índice de Precios de Consumo Armonizado'],
                      ['IPCA1885','Índice de Precios de Consumo Armonizado'],
                      ['IPV769','Índice de Precios de Vivienda'],
                      ['IPV948','Índice de Precios de Vivienda'],
                      ['IPV949','Índice de Precios de Vivienda'],
                      ['IPI10277','Índice de producción industrial'],
                      ['IPI10276','Índice de producción industrial'],
                      ['ICNE91','Índice de cifra de negocios empresarial']]

3. Ejecucción extracción series INE

In [4]:
# MAPA_Periodicidad = requests.get('https://servicios.ine.es/wstempus/js/ES/PERIODICIDADES?page=1').json()

first = True

for i in LISTA_SERIES_INE:
#    print(i)
    DATOS_SERIE_I = requests.get(url_datos.format(codigo=i[0],FECHA_INI=FECHA_INI,FECHA_FIN=FECHA_FIN)).json()
    METADATOS_SERIE_I = requests.get(url_metadatos.format(codigo=i[0])).json()
    
    API_TXT = url_datos.format(codigo=i[0],FECHA_INI=FECHA_INI,FECHA_FIN=FECHA_FIN)
    CODIGO_SERIE = DATOS_SERIE_I['COD']
    NOMBRE_SERIE = METADATOS_SERIE_I['Nombre'].upper()
    TABLA_SERIE = i[1]
    FUENTE_SERIE = 'INE'
    
    
    if 'DATO BASE' in NOMBRE_SERIE and 'VARIACIÓN TRIMESTRAL' not in NOMBRE_SERIE and 'VARIACIÓN ANUAL' not in NOMBRE_SERIE:
        TIPO_DATO = 'DATO BASE'
    elif 'DATO BASE' not in NOMBRE_SERIE and 'VARIACIÓN TRIMESTRAL' in NOMBRE_SERIE and 'VARIACIÓN ANUAL' not in NOMBRE_SERIE:
        TIPO_DATO = 'VARIACIÓN TRIMESTRAL'
    elif 'DATO BASE' not in NOMBRE_SERIE and 'VARIACIÓN TRIMESTRAL' not in NOMBRE_SERIE and 'VARIACIÓN ANUAL' in NOMBRE_SERIE:
        TIPO_DATO = 'VARIACIÓN ANUAL'
    else:
        TIPO_DATO = 'DATO BASE'
    
#     Extraemos la variable fecha (//100 por venir en UNIX milliseconds)
    FECHA_1 = [date.fromtimestamp(x['Fecha'] // 1000) for x in DATOS_SERIE_I['Data']] 
    
#     Variable fecha a fin de mes / fin de trimestre
    if 'Q' in pd.infer_freq(pd.to_datetime(FECHA_1)):
        Periodicidad = 'Trimestral'
    elif 'M' in pd.infer_freq(pd.to_datetime(FECHA_1)):
        Periodicidad = 'Mensual'
    else:
        Periodicidad = pd.infer_freq(pd.to_datetime(FECHA_1))
    
#    Fromateamos la variable fecha a fin de mes / fin de trimestre y sacamos max y min

    if Periodicidad == 'Trimestral':
        FECHA_DATO = pd.Index(pd.to_datetime(FECHA_1)).to_period('Q').to_timestamp('Q')
    elif Periodicidad == 'Mensual':
        FECHA_DATO = pd.Index(pd.to_datetime(FECHA_1)).to_period('M').to_timestamp('M')
    else:
        pass # Hay error, frequencia no considerada en el código
   
    MIN_DATE = str(min(FECHA_DATO.date))
    MAX_DATE = str(max(FECHA_DATO.date))
    
#    Sacamos los datos y los juntamos con la fecha, si es mensual la pasamos a trimestral mirando mes 3,6,9,12

    VALOR_DATO = [x['Valor'] for x in DATOS_SERIE_I['Data']]
    
    tabla = pd.DataFrame(VALOR_DATO, 
                     index=FECHA_DATO.date,
                     columns=[CODIGO_SERIE])
    DATOS_I_DF_P = pd.DataFrame(VALOR_DATO, index=FECHA_DATO.date, columns=[CODIGO_SERIE])
    
    if first:
        first = False
        DATOS_INE = pd.DataFrame(DATOS_I_DF_P[CODIGO_SERIE].where(pd.DatetimeIndex(DATOS_I_DF_P.index).month.isin([3,6,9,12]))).dropna()
        METADATOS_INE = pd.DataFrame({'Código Serie': [DATOS_SERIE_I['COD']],
                                      'Tabla Serie': [i[1]],
                                      'Fuente Serie': ['INE'],
                                      'Periodicidad máxima': [Periodicidad],
                                      'Tipo dato': [TIPO_DATO],
                                      'Inicio Serie': [MIN_DATE],
                                      'Fin Serie': [MAX_DATE],
                                      'Nombre Serie': [DATOS_SERIE_I['Nombre']],
                                      'Comentarios' : [''],
                                      'Link API':[API_TXT],
                                      'Fecha consulta' :[FECHA_FIN]})
    else:
        DATOS_I_DF = pd.DataFrame(DATOS_I_DF_P[CODIGO_SERIE].where(pd.DatetimeIndex(DATOS_I_DF_P.index).month.isin([3,6,9,12]))).dropna()
        DATOS_INE = DATOS_INE.join(DATOS_I_DF, how='outer')
        
        METADATOS_I = pd.DataFrame({'Código Serie': [DATOS_SERIE_I['COD']],
                                      'Tabla Serie': [i[1]],
                                      'Fuente Serie': ['INE'],
                                      'Periodicidad máxima': [Periodicidad],
                                      'Tipo dato': [TIPO_DATO],
                                      'Inicio Serie': [MIN_DATE],
                                      'Fin Serie': [MAX_DATE],
                                      'Nombre Serie': [DATOS_SERIE_I['Nombre']],
                                      'Comentarios' : [''],
                                      'Link API':[API_TXT],
                                      'Fecha consulta' :[FECHA_FIN]})
        METADATOS_INE = pd.concat([METADATOS_INE,METADATOS_I])

3. Extracción Series BDE

In [5]:
locale.setlocale(locale.LC_ALL, 'es_ES') 
first = True

for m in TABLAS_SERIES_BDE:
#    print(m)
    TABLA_BDE_IT = pd.read_csv(m,encoding='latin-1',header=None)
    
    # Miramos que series están en la lista para la tabla m
    L_SBDE_IT = TABLA_BDE_IT[TABLA_BDE_IT[0]=='NÚMERO SECUENCIAL'].astype('string')
    L_SBDE_IT = np.append(['NÚMERO SECUENCIAL'],
                          L_SBDE_IT[L_SBDE_IT.isin(LISTA_SERIES_BDE)].dropna(axis=1).reset_index(drop=True).iloc[0].values)
    
    # Filtramos series que estén en la lista
    TABLA_BDE_IT_2 = TABLA_BDE_IT.rename(columns=TABLA_BDE_IT[TABLA_BDE_IT[0]=='NÚMERO SECUENCIAL'].reset_index().T[0].astype('string'))    
    TABLA_BDE_IT_2 = TABLA_BDE_IT_2[L_SBDE_IT]     
   
    # Metadatos
    CODIGOS = TABLA_BDE_IT_2.columns.values[1:]
    TABLAS = [x.split('.',1)[0] for x in TABLA_BDE_IT_2[TABLA_BDE_IT_2['NÚMERO SECUENCIAL']=='ALIAS DE LA SERIE'].values[0]][1:]
    FUENTE_SERIE =['BDE' for x in TABLAS]
    NOMBRE_SERIE = TABLA_BDE_IT_2[TABLA_BDE_IT_2['NÚMERO SECUENCIAL']=='DESCRIPCIÓN DE LA SERIE'].values[0][1:]
    TIPO_DATO =['' for x in TABLAS]
    COMENTARIOS =['' for x in TABLAS]
    LINK_API = [m for x in TABLAS]
    FECHA_CONSULTA = [FECHA_FIN for x in TABLAS]
    # Datos
    TABLA_BDE_IT_2 = TABLA_BDE_IT_2.rename(columns={"NÚMERO SECUENCIAL": "FECHA"})
        # Borrar filas donde no hay datos
    F_IT_L = []
    for k in TABLA_BDE_IT_2['FECHA'].values:   
        try:
            F_IT_V = dt.datetime.strptime(k.split(' ')[0]+'.'+k.split(' ')[1],'%b%Y')
        except:
            F_IT_V = 'DROP' 
        F_IT_L.append(F_IT_V)
    
    TABLA_BDE_IT_2['FECHA']=F_IT_L
    TABLA_BDE_IT_2 = TABLA_BDE_IT_2[TABLA_BDE_IT_2.FECHA != 'DROP']
    TABLA_BDE_IT_2['FECHA'] = pd.Index(pd.to_datetime(TABLA_BDE_IT_2['FECHA'])).to_period('M').to_timestamp('M')
    
        # Borrar filas donde no hay ningún valor informado para ninguna variable 
    TABLA_BDE_IT_2 = TABLA_BDE_IT_2.replace('_', np.NaN)
    TABLA_BDE_IT_2 = TABLA_BDE_IT_2.set_index('FECHA')
    TABLA_BDE_IT_2 = TABLA_BDE_IT_2.dropna(axis=0,how='all')
    
    
    MAX_DATES = [max(TABLA_BDE_IT_2[h].dropna().index) for h in TABLA_BDE_IT_2.columns]
    MAX_DATES = [x.date() for x in MAX_DATES]
    MIN_DATES = [min(TABLA_BDE_IT_2[h].dropna().index) for h in TABLA_BDE_IT_2.columns]
    MIN_DATES = [x.date() for x in MIN_DATES]
    PERIOD = ['Trimestral' if 'Q' in pd.infer_freq(pd.to_datetime(TABLA_BDE_IT_2[i].dropna().index)) 
               else 'Mensual' if 'M' in pd.infer_freq(pd.to_datetime(TABLA_BDE_IT_2[i].dropna().index))
               else pd.infer_freq(pd.to_datetime(TABLA_BDE_IT_2[i].dropna().index)) for i in TABLA_BDE_IT_2.columns]
    # Merge
    if first:
        first = False
        DATOS_BDE = TABLA_BDE_IT_2[pd.DatetimeIndex(TABLA_BDE_IT_2.index).month.isin([3,6,9,12])].dropna(axis=0,how='all')
        METADATOS_BDE = pd.DataFrame({'Código Serie': CODIGOS,
                                      'Tabla Serie': TABLAS,
                                      'Fuente Serie': FUENTE_SERIE,
                                      'Periodicidad máxima': PERIOD,
                                      'Tipo dato': TIPO_DATO,
                                      'Inicio Serie': MIN_DATES,
                                      'Fin Serie': MAX_DATES,
                                      'Nombre Serie': NOMBRE_SERIE,
                                      'Comentarios' : COMENTARIOS,
                                      'Link API':LINK_API,
                                      'Fecha consulta' :FECHA_CONSULTA})
    else:

        DATOS_BDE = DATOS_BDE.join(TABLA_BDE_IT_2[pd.DatetimeIndex(TABLA_BDE_IT_2.index).month.isin([3,6,9,12])].dropna(axis=0,how='all'), how='outer')
        
        METADATOS_I = pd.DataFrame({'Código Serie': CODIGOS,
                                      'Tabla Serie': TABLAS,
                                      'Fuente Serie': FUENTE_SERIE,
                                      'Periodicidad máxima': PERIOD,
                                      'Tipo dato': TIPO_DATO,
                                      'Inicio Serie': MIN_DATES,
                                      'Fin Serie': MAX_DATES,
                                      'Nombre Serie': NOMBRE_SERIE,
                                      'Comentarios' : COMENTARIOS,
                                      'Link API':LINK_API,
                                      'Fecha consulta' :FECHA_CONSULTA})
        METADATOS_BDE = pd.concat([METADATOS_BDE,METADATOS_I])

DATOS_BDE = DATOS_BDE.apply(pd.to_numeric)

4. Extracción ECB DATA WAREHOUSE

In [6]:
first = True

for i in LISTA_SERIES_ECB:
        DATOS_SERIE_I = pd.read_csv(io.StringIO(requests.get(url_ecb.format(FLOWREF=i[0],KEY=i[1])).text))

        DATOS_SERIE_I = DATOS_SERIE_I[['TIME_PERIOD','OBS_VALUE','TITLE_COMPL']]
        
        # Medtadatos
        NOMBRE_SERIE = DATOS_SERIE_I['TITLE_COMPL'][0]
        CODIGO_SERIE = i[0]+'.'+i[1]
        TABLA_SERIE = ''
        FUENTE_SERIE = 'ECB'
        TIPO_DATO = ''
        COMENTARIOS = ''
        LINK_API = url_ecb.format(FLOWREF=i[0],KEY=i[1])
        
        #Datos
        DATOS_SERIE_I = DATOS_SERIE_I[['TIME_PERIOD','OBS_VALUE']].dropna()
        DATOS_SERIE_I['TIME_PERIOD']=[dt.datetime.strptime(i,'%Y-%m') for i in DATOS_SERIE_I['TIME_PERIOD']]
        DATOS_SERIE_I['TIME_PERIOD'] = pd.Index(pd.to_datetime(DATOS_SERIE_I['TIME_PERIOD'])).to_period('M').to_timestamp('M')
        DATOS_SERIE_I = DATOS_SERIE_I.set_index('TIME_PERIOD')
        DATOS_SERIE_I = DATOS_SERIE_I.rename(columns={"OBS_VALUE": CODIGO_SERIE})
    
        MAX_DATES = max(DATOS_SERIE_I.index)
        MIN_DATES = min(DATOS_SERIE_I.index)
        
        if 'Q' in pd.infer_freq(pd.to_datetime(DATOS_SERIE_I.index)):
            Periodicidad = 'Trimestral'
        elif 'M' in pd.infer_freq(pd.to_datetime(DATOS_SERIE_I.index)):
            Periodicidad = 'Mensual'
        else:
            Periodicidad = pd.infer_freq(pd.to_datetime(DATOS_SERIE_I.index))
         
        if first:
            first = False
            DATOS_ECB = DATOS_SERIE_I[pd.DatetimeIndex(DATOS_SERIE_I.index).month.isin([3,6,9,12])].dropna(axis=0,how='all')
            METADATOS_ECB = pd.DataFrame({'Código Serie': [CODIGO_SERIE],
                                      'Tabla Serie': [TABLA_SERIE],
                                      'Fuente Serie': [FUENTE_SERIE],
                                      'Periodicidad máxima': [Periodicidad],
                                      'Tipo dato': [TIPO_DATO],
                                      'Inicio Serie': [MIN_DATES],
                                      'Fin Serie': [MAX_DATES],
                                      'Nombre Serie': [NOMBRE_SERIE],
                                      'Comentarios' : [COMENTARIOS],
                                      'Link API':[LINK_API],
                                      'Fecha consulta' :[FECHA_FIN]})
        else:
            DATOS_E_DF = DATOS_SERIE_I[pd.DatetimeIndex(DATOS_SERIE_I.index).month.isin([3,6,9,12])].dropna(axis=0,how='all')
            DATOS_ECB = DATOS_ECB.join(DATOS_E_DF, how='outer')
        
            METADATOS_I = pd.DataFrame({'Código Serie': [CODIGO_SERIE],
                                      'Tabla Serie': [TABLA_SERIE],
                                      'Fuente Serie': [FUENTE_SERIE],
                                      'Periodicidad máxima': [Periodicidad],
                                      'Tipo dato': [TIPO_DATO],
                                      'Inicio Serie': [MIN_DATES],
                                      'Fin Serie': [MAX_DATES],
                                      'Nombre Serie': [NOMBRE_SERIE],
                                      'Comentarios' : [COMENTARIOS],
                                      'Link API':[LINK_API],
                                      'Fecha consulta' :[FECHA_FIN]})
            METADATOS_ECB = pd.concat([METADATOS_ECB,METADATOS_I])

5. Series manuales

5. Juntar datasets

In [7]:
METADATOS_FINAL = pd.concat([METADATOS_INE,METADATOS_BDE,METADATOS_ECB])
METADATOS_FINAL = pd.merge(METADATOS_FINAL, pd.DataFrame.from_records(COMENTARIOS_SERIES), how='left', left_on='Código Serie', right_on=0)
METADATOS_FINAL = METADATOS_FINAL.assign(Comentarios=METADATOS_FINAL[1]).drop([0,1],axis=1);
DATOS_F1 = DATOS_INE.join(DATOS_BDE.join(DATOS_ECB, how='outer'), how='outer')
DATOS_F2 = DATOS_F1[pd.DatetimeIndex(DATOS_F1.index).year >= int(FECHA_INI[0:4])].dropna(axis=0,how='all')

In [8]:
ruta_metadatos = r'C:\Users\Usuari\Desktop\SeriesMacro\{FECHA}_METADATA_TIME_SERIES.xlsx'
ruta_datos = r'C:\Users\Usuari\Desktop\SeriesMacro\{FECHA}_TIME_SERIES.xlsx'

METADATOS_FINAL.to_excel (ruta_metadatos.format(FECHA=FECHA_FIN))
DATOS_F2.to_excel (ruta_datos.format(FECHA=FECHA_FIN))